# For Checking the Listing Status


In [2]:
import os
import sys
import pathlib
import requests
import aiohttp
import asyncio
import random
import pygsheets
from typing import List, Tuple
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import proxyscrape



In [3]:
SHEETS_SECRET = "i-tec-349718-b1c6f340ebe6.json"
SHEETS_ID = "1otu2JmIGmjM6dbEWuO7dqx8yiq_p1ocjFwUK7HOWjoo"

In [4]:
path = !pwd
ROOT = os.path.join(*os.path.split(path[0])[:-1])

In [5]:


sys.path.append(ROOT)


In [6]:
from fantastic_amazon.sheets_connector import GoogleSheets



In [7]:
sheet = GoogleSheets(sheet_key=SHEETS_ID , sheet_name="Sheet1" , service_file=os.path.join(ROOT, SHEETS_SECRET))

In [8]:
urls = [(row, (row[3].hyperlink).split("?")[0]) for row in sheet.w.range("A:L")[1:] if row[0].value != '']

In [9]:
urls = [row for row in urls if row[1] != '']

In [10]:
STORE_ID = "F7A5C936-4824-42B0-8265-64E36DE5D115"

def check_store(row: Tuple[List[pygsheets.Cell], str], html: BeautifulSoup):
    if STORE_ID in html.body.find(id="bylineInfo")['href']:
        row[0][7].set_value('Y')
    else:
        row[0][7].set_value('N')
    row[0][7].update()

def aplus(row: Tuple[List[pygsheets.Cell], str], html: BeautifulSoup):
    if html.body.find(id="aplus"):
        row[0][8].set_value('Y')
    else:
        row[0][8].set_value('N')
    row[0][8].update()


def video(row: Tuple[List[pygsheets.Cell], str], html: BeautifulSoup):
    if 'video'in html.body.find(id="altImages").text.strip().lower():
        row[0][9].set_value('Y')
    else:
        row[0][9].set_value('N')
    row[0][8].update()


def pn(row: Tuple[List[pygsheets.Cell], str], html: BeautifulSoup):
    if row[0][0].value_unformatted.lower() in html.body.find(id="productDetails_detailBullets_sections1").text.lower():
        row[0][11].set_value('Y')
    else:
        row[0][11].set_value('N')
    row[0][11].update()

In [12]:
headers = {
  'Cookie': 'i18n-prefs=USD'
}


CHECK_FUNCTIONS = [check_store, aplus, video, pn]


async def parse_html(session, row):
  headers = {'User-Agent':str(UserAgent().random)}
  async with session.post(row[1], headers=headers) as resp:
      # return await resp.text()
      html = BeautifulSoup(await resp.text(), )
      for fn in CHECK_FUNCTIONS:
        fn(row, html)
  print("Done with ", row[1])



In [13]:

data = []
tasks = []

async with aiohttp.ClientSession() as session:
    
    for url in urls:
        tasks.append(asyncio.ensure_future(parse_html(session, url)))

    data = await asyncio.gather(*tasks)

Done with  http://www.amazon.com/dp/B09THLTQYR
Done with  https://www.amazon.com/dp/B09THKL5D2
Done with  http://www.amazon.com/dp/B08CVSWZ3H
Done with  http://www.amazon.com/dp/B08NK1QKZ5
Done with  http://www.amazon.com/dp/B09C55JB4Y
Done with  http://www.amazon.com/dp/B08CVS9TGQ
Done with  https://www.amazon.com/dp/B09THK9TH6
Done with  http://www.amazon.com/dp/B09K4GKHWZ
Done with  https://www.amazon.com/dp/B086T5QSDK
Done with  https://www.amazon.com/dp/B09YT5PMGL
Done with  http://www.amazon.com/dp/B07MHVN9SV
Done with  http://www.amazon.com/dp/B09C53JG5H
Done with  http://www.amazon.com/dp/B09C537GHV
Done with  http://www.amazon.com/dp/B09C53H8QB
Done with  http://www.amazon.com/dp/B0768CY3WL
Done with  http://www.amazon.com/dp/B075Z5WK7S
